In [150]:
import pandas as pd

In [151]:
df = pd.read_csv('optimized_truck_schedules.csv')

In [152]:
df

,truck_id,route_id,fuel_type,distance,departure_date,load_capacity,route_category,mileage_mpg,average_hours,average_speed_mph,reassigned
0,26392203.0,R-5eb1238b,gas,212.35,2019-01-16,3000.0,Short,15.0,170.93,39.15,True
1,26392203.0,R-ceabd159,gas,222.41,2019-02-03,3000.0,Short,15.0,170.93,39.15,True
2,26392203.0,R-56cddd65,gas,239.78,2019-01-25,3000.0,Short,15.0,170.93,39.15,True
3,26392203.0,R-a6faaa76,gas,335.83,2019-02-09,3000.0,Short,15.0,170.93,39.15,True
4,26392203.0,R-4f913e38,gas,368.04,2019-01-01,3000.0,Short,15.0,170.93,39.15,True
...,...,...,...,...,...,...,...,...,...,...,...
11338,25681071.0,R-fa6a340a,diesel,10012.40,2019-01-01,20000.0,Long,24.0,200.25,50.43,False
11339,90639626.0,R-fa6a340a,diesel,10012.40,2019-02-06,3000.0,Long,23.0,200.25,44.33,False
11340,12586136.0,R-f21e668c,diesel,10055.01,2019-01-19,15000.0,Long,22.0,201.10,40.55,False
11341,42562600.0,R-93d99a8b,diesel,10055.01,2019-02-06,15000.0,Long,23.0,201.10,60.73,False


In [153]:
df = df.drop(columns = 'reassigned')

In [154]:
fuel_used = (df['distance']/df['mileage_mpg']).sum()
fuel_used

np.float64(665346.261164699)

In [155]:
total_distance_before = df.groupby(['load_capacity','fuel_type'])['distance'].sum()
total_distance_before

load_capacity  fuel_type
3000.0         diesel       2326111.39
               gas          1070897.23
4000.0         diesel       1670437.00
               gas           693748.86
6000.0         diesel       1463618.89
               gas           735320.76
10000.0        diesel       1122516.12
               gas           705371.84
15000.0        diesel       2187778.53
               gas          1169145.44
20000.0        diesel        705204.37
               gas           277340.59
Name: distance, dtype: float64

In [156]:
import pandas as pd

# Grouping by load_capacity and fuel_type
grouped = df.groupby(['load_capacity', 'fuel_type'])

# Store initial total distance for each (load_capacity, fuel_type) to verify correctness later
initial_distance_sums = df.groupby(['load_capacity', 'fuel_type'])['distance'].sum().reset_index()
initial_distance_sums.rename(columns={'distance': 'initial_total_distance'}, inplace=True)

# Initialize reassigned column
df['reassigned'] = False

for (load_capacity, fuel_type), group in grouped:
    # Compute total distance per truck in the group
    truck_distances = group.groupby('truck_id')['distance'].sum().reset_index()
    
    # Merge back with original group
    group = group.merge(truck_distances, on='truck_id', suffixes=('', '_total'))
    
    # Sort trucks by mpg (ascending) and total_distance (descending)
    sorted_trucks = group[['truck_id', 'mileage_mpg', 'distance_total']].drop_duplicates()
    sorted_trucks = sorted_trucks.sort_values(by=['mileage_mpg', 'distance_total'], ascending=[True, False])
    
    # Convert to list of tuples for iteration
    truck_list = sorted_trucks[['truck_id', 'mileage_mpg', 'distance_total']].values.tolist()

    swaps_made = True  # Flag to check if swaps happen
    while swaps_made:
        swaps_made = False  # Reset flag before checking
        
        for i in range(len(truck_list)):
            for j in range(i + 1, len(truck_list)):  # Compare with every next truck in the list
                truck_low_mpg, mpg_low, dist_low = truck_list[i]
                truck_high_mpg, mpg_high, dist_high = truck_list[j]

                # Skip if MPG is the same (no benefit from swapping)
                if mpg_low == mpg_high:
                    continue  

                # Swap schedules if higher MPG truck has a shorter distance
                if mpg_high > mpg_low and dist_high < dist_low:
                    # Get the indices of trips for each truck
                    truck_low_rows = df[(df['truck_id'] == truck_low_mpg) & 
                                        (df['load_capacity'] == load_capacity) & 
                                        (df['fuel_type'] == fuel_type)].index
                    truck_high_rows = df[(df['truck_id'] == truck_high_mpg) & 
                                         (df['load_capacity'] == load_capacity) & 
                                         (df['fuel_type'] == fuel_type)].index

                    # Ensure we only swap up to the minimum number of trips
                    min_len = min(len(truck_low_rows), len(truck_high_rows))
                    truck_low_rows = truck_low_rows[:min_len]
                    truck_high_rows = truck_high_rows[:min_len]

                    # Make a copy before swapping to ensure safe assignments
                    low_values = df.loc[truck_low_rows, ['distance', 'departure_date', 'route_id']].copy()
                    high_values = df.loc[truck_high_rows, ['distance', 'departure_date', 'route_id']].copy()

                    # Swap safely
                    df.loc[truck_low_rows, ['distance', 'departure_date', 'route_id']] = high_values.values
                    df.loc[truck_high_rows, ['distance', 'departure_date', 'route_id']] = low_values.values

                    # Mark as reassigned
                    df.loc[df['truck_id'] == truck_low_mpg, 'reassigned'] = True
                    df.loc[df['truck_id'] == truck_high_mpg, 'reassigned'] = True

                    # Update truck_list with new distances after swapping
                    truck_list[i] = (truck_low_mpg, mpg_low, dist_high)
                    truck_list[j] = (truck_high_mpg, mpg_high, dist_low)

                    swaps_made = True  # A swap occurred, so we should continue checking

# Verify if total distances remain unchanged after reassignment
final_distance_sums = df.groupby(['load_capacity', 'fuel_type'])['distance'].sum().reset_index()
final_distance_sums.rename(columns={'distance': 'final_total_distance'}, inplace=True)

# Compare initial vs final
distance_check = initial_distance_sums.merge(final_distance_sums, on=['load_capacity', 'fuel_type'])
distance_check['difference'] = distance_check['final_total_distance'] - distance_check['initial_total_distance']

print(distance_check)  # Should be all zeros if correct
print("Reassignment Complete!")


    load_capacity fuel_type  initial_total_distance  final_total_distance  \
0          3000.0    diesel              2326111.39            2326111.39   
1          3000.0       gas              1070897.23            1070897.23   
2          4000.0    diesel              1670437.00            1670437.00   
3          4000.0       gas               693748.86             693748.86   
4          6000.0    diesel              1463618.89            1463618.89   
5          6000.0       gas               735320.76             735320.76   
6         10000.0    diesel              1122516.12            1122516.12   
7         10000.0       gas               705371.84             705371.84   
8         15000.0    diesel              2187778.53            2187778.53   
9         15000.0       gas              1169145.44            1169145.44   
10        20000.0    diesel               705204.37             705204.37   
11        20000.0       gas               277340.59             277340.59   

In [157]:
df.groupby(['load_capacity', 'mileage_mpg', 'fuel_type'])['distance'].sum().reset_index().sort_values(['load_capacity', 'fuel_type', 'mileage_mpg']).head(20)


,load_capacity,mileage_mpg,fuel_type,distance
5,3000.0,18.0,diesel,6320.72
7,3000.0,19.0,diesel,23840.78
9,3000.0,20.0,diesel,37017.01
11,3000.0,21.0,diesel,132597.15
13,3000.0,22.0,diesel,286599.06
15,3000.0,23.0,diesel,329735.33
17,3000.0,24.0,diesel,517925.95
19,3000.0,25.0,diesel,426323.05
20,3000.0,26.0,diesel,384985.48
21,3000.0,27.0,diesel,77830.42


In [158]:
df.sort_values(by = ['truck_id','departure_date'],ascending = False)

,truck_id,route_id,fuel_type,distance,departure_date,load_capacity,route_category,mileage_mpg,average_hours,average_speed_mph,reassigned
4170,99981667.0,R-29e0817a,gas,191.76,2019-02-12,4000.0,Short,14.0,2.53,61.52,True
4172,99981667.0,R-7fe6fd46,gas,287.28,2019-02-09,4000.0,Short,14.0,2.53,61.52,True
4171,99981667.0,R-48bf5cc8,gas,267.04,2019-02-06,4000.0,Short,14.0,2.53,61.52,True
4166,99981667.0,R-81a8f294,gas,46.19,2019-02-03,4000.0,Short,14.0,2.53,61.52,True
4176,99981667.0,R-0ccbf205,gas,768.78,2019-02-03,4000.0,Short,14.0,2.53,61.52,True
...,...,...,...,...,...,...,...,...,...,...,...
10688,10008392.0,R-4d5c4624,diesel,1912.18,2019-02-02,10000.0,Long,23.0,50.26,60.74,True
10977,10008392.0,R-4283e577,diesel,2707.17,2019-01-25,10000.0,Long,23.0,59.92,60.74,True
10974,10008392.0,R-8f148291,diesel,2081.68,2019-01-17,10000.0,Long,23.0,42.61,60.74,True
10910,10008392.0,R-e8a0f522,diesel,1968.09,2019-01-09,10000.0,Long,23.0,58.96,60.74,True


In [159]:
new_fuel_used = (df['distance'] / df['mileage_mpg']).sum()
new_fuel_used

np.float64(656923.6456697921)

In [160]:
total_distance_after = df.groupby(['load_capacity','fuel_type'])['distance'].sum()
total_distance_after

load_capacity  fuel_type
3000.0         diesel       2326111.39
               gas          1070897.23
4000.0         diesel       1670437.00
               gas           693748.86
6000.0         diesel       1463618.89
               gas           735320.76
10000.0        diesel       1122516.12
               gas           705371.84
15000.0        diesel       2187778.53
               gas          1169145.44
20000.0        diesel        705204.37
               gas           277340.59
Name: distance, dtype: float64

In [161]:
df.to_csv('optimized_truck_schedules_mpg.csv')